In [357]:
from pathlib import Path
import configparser
from psycopg2 import connect
import pandas.io.sql as pandasql
from IPython.display import HTML
from psycopg2.extras import execute_values
import requests
from psycopg2.extras import RealDictCursor
import json
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [496]:
# Pull data from Database
count_sql = '''SELECT COUNT(*) 
                FROM gis.centreline 
                WHERE FCODE_DESC IN ('Collector','Collector Ramp',
                'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
                'Minor Arterial Ramp','Pending') '''
with con:
    with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(count_sql)
            count_results = cur.fetchall()
            count = count_results[0]['count']
            
unmatched_gid = []

for i in range(0, count, 500):
    get_centreline_sql = '''SELECT 'Feature' as type,  json_build_object('gid', gid) as properties, ST_AsGeoJson(geom)::json as geometry FROM gis.centreline WHERE FCODE_DESC IN ('Collector','Collector Ramp',
    'Local','Major Arterial','Major Arterial Ramp','Minor Arterial',
    'Minor Arterial Ramp','Pending') ORDER BY gid LIMIT {} OFFSET {}'''.format(500, i)

    with con:
        with con.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(get_centreline_sql)
            results = cur.fetchall()
            final_json = json.dumps({"type": "FeatureCollection",
                                    "features": results})
            
# Send data to SharedStreets API    
    base_url = "https://api.sharedstreets.io/v0.1.0/match/geoms"
    apiKey = "bdd23fa1-7ac5-4158-b354-22ec946bb575"
    params = {'ignoreDirection': 'false',
              'bearingTolerance' : 45 ,
              'searchRadius' : 60,
              'auth' : apiKey,
              'tileHierarchy': 6,
              'dataSource': 'osm/planet-181029',
              'lengthTolerance' : 0.60,
              'snapTopology': 'true'
             }
    headers= {"Content-type": "application/json; charset=UTF-8"}
    
    r = requests.post(base_url, params = params, headers = headers, json = {"type": "FeatureCollection", "features": results})
    returned_json = r.json()
    feature = returned_json.get('matched').get('features')[0]
    unmatched = returned_json.get('unmatched').get('features')
    for feature in unmatched:
        unmatched_id = (feature['properties']['gid'])
        unmatched_gid.append(unmatched_id)
    
# Send row to database
    features = returned_json.get('matched').get('features')
    rows = []
    for feature in features:
           
        if  feature['properties']['section'][0] > feature['properties']['section'][1]:
            print("Error with gid {}".format(feature['properties']['originalFeature']['properties']['gid']))
            print(feature)
            feature['properties']['section'][0] = 0
            feature['properties']['section'][1] = 0
            
        row = (feature['properties']['referenceId'], 
                   feature['properties']['fromIntersectionId'],
                   feature['properties']['toIntersectionId'],
                   feature['properties']['roadClass'], 
                   feature['properties']['direction'],
                   feature['properties']['referenceLength'],
                   feature['properties']['side'], 
                   feature['properties'].get('score', None),
                   '[' + str(feature['properties']['section'][0]) +', '+ str(feature['properties']['section'][1]) +')', 
                   'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['coordinates'])+')', 
                   feature['properties']['originalFeature']['properties']['gid'])
   

        rows.append(row)
        
        

    sql='INSERT INTO natalie.test_table (reference_id, from_intersection, to_intersection, road_class, direction, reference_length, side,score, section, geometry, gid) VALUES %s'

    with con:
        with con.cursor() as cur:
            execute_values(cur, 
                           sql, 
                           rows) 
            
# Print unmatched features                
print(unmatched_gid)

Error with gid 94707
{'geometry': {'coordinates': [[-79.42909039317898, 43.732976647925824], [-79.42909030249221, 43.73297643392493]], 'type': 'LineString'}, 'type': 'Feature', 'properties': {'roadClass': 'Residential', 'section': [101.86488640123036, 101.84], 'toIntersectionId': '809c0ef627da50956544354a0a4512bc', 'side': 'unknown', 'originalFeature': {'geometry': {'type': 'LineString', 'coordinates': [[-79.42909106, 43.732976508], [-79.42947136, 43.733836421]]}, 'type': 'Feature', 'properties': {'gid': 94707}}, 'referenceId': 'ad8e662c53973db9ea0253601fa8a433', 'fromIntersectionId': '9fc8d55d112a88116dc75e05dcebd7ce', 'score': 1.7763916974183696, 'referenceLength': 101.84, 'direction': 'forward', 'geometryId': '24d6d4ffb8b610cf248f2fdb28327025'}}
Error with gid 95569
{'geometry': {'coordinates': [[-79.40233711367165, 43.725681775084425], [-79.40233710466494, 43.72568174118564]], 'type': 'LineString'}, 'type': 'Feature', 'properties': {'roadClass': 'Secondary', 'section': [63.35383823

In [494]:
len(unmatched_gid)

2182